# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [1]:
import requests #Để gọi API
import re #Để xử lý data dạng string
from datetime import datetime as dt #Để xử lý data dạng datetime
!pip install gspread as gspread
#Để update data lên Google Spreadsheet
!pip install gspread_dataframe 
from gspread_dataframe import set_with_dataframe 
#Để update data lên Google Spreadsheet
import pandas as pd # Để update data dạng bản
import json 
!pip install oauth2client
from oauth2client.service_account import ServiceAccountCredentials 
#Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [16]:
!dir

 Volume in drive C is System
 Volume Serial Number is 140C-D8FD

 Directory of c:\Users\Admin\OneDrive\Documents\GitHub\atom-assignments\assignment_3

05/19/2021  11:19 PM    <DIR>          .
05/19/2021  11:19 PM    <DIR>          ..
05/21/2021  12:11 AM               224 env_variable.json
05/20/2021  08:14 PM            82,711 home_assignment_3.ipynb
05/19/2021  11:19 PM             2,326 my_project_1.json
05/18/2021  09:59 PM               238 README.md
               4 File(s)         85,499 bytes
               2 Dir(s)  10,429,677,568 bytes free


In [17]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)

In [18]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [19]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
response_json

  'image_192': 'https://avatars.slack-edge.com/2021-04-23/2003495440980_4ae805304d8761161cf6_192.png',
    'image_512': 'https://avatars.slack-edge.com/2021-04-23/2003495440980_4ae805304d8761161cf6_512.png',
    'image_1024': 'https://avatars.slack-edge.com/2021-04-23/2003495440980_4ae805304d8761161cf6_1024.png',
    'status_text_canonical': '',
    'team': 'T01B7SGGMLJ'},
   'is_admin': False,
   'is_owner': False,
   'is_primary_owner': False,
   'is_restricted': False,
   'is_ultra_restricted': False,
   'is_bot': False,
   'is_app_user': False,
   'updated': 1619221313,
   'is_email_confirmed': True},
  {'id': 'U01VBGPP1EU',
   'team_id': 'T01B7SGGMLJ',
   'name': 'tranctan96',
   'deleted': False,
   'color': 'bc3663',
   'real_name': 'Tân Trần',
   'tz': 'Asia/Bangkok',
   'tz_label': 'Indochina Time',
   'tz_offset': 25200,
   'profile': {'title': '',
    'phone': '',
    'skype': '',
    'real_name': 'Tân Trần',
    'real_name_normalized': 'Tan Trần',
    'display_name': 'Tân T

In [20]:
user_dat = response_json['members']

### TODO #1
Hoàn tất đoạn code sau

In [21]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [22]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [23]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](../img/google_service_account.png)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](../img/gservice_acc_json.png)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](../img/enable_api.png)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [24]:
!dir

 Volume in drive C is System
 Volume Serial Number is 140C-D8FD

 Directory of c:\Users\Admin\OneDrive\Documents\GitHub\atom-assignments\assignment_3

05/19/2021  11:19 PM    <DIR>          .
05/19/2021  11:19 PM    <DIR>          ..
05/21/2021  12:11 AM               224 env_variable.json
05/20/2021  08:14 PM            82,711 home_assignment_3.ipynb
05/19/2021  11:19 PM             2,326 my_project_1.json
05/18/2021  09:59 PM               238 README.md
               4 File(s)         85,499 bytes
               2 Dir(s)  10,429,116,416 bytes free


In [25]:
import gspread
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'my_project_1.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](../img/enable_api.png)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](../img/add_gsheet.png)

In [16]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1nNlQbhJbdfJLWCYirnHdOSB6i9KNFVPZe1qNTWkgk4o' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](../img/slack_user_gsheet.png)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [26]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [27]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01B4PVGLVB',
 'name': 'general',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1600856703,
 'is_archived': False,
 'is_general': True,
 'unlinked': 0,
 'name_normalized': 'general',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01BE2PR6LU',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.',
  'creator': 'U01BE2PR6LU',
  'last_set': 1600856703},
 'previous_names': [],
 'num_members': 63}

### TODO#3 
* Tìm id của channel #atom-assignment2

In [28]:
for i in range(len(channel_ls)): #Tìm trong các phần tử của channel_ls
    if channel_ls[i]['name'] == 'atom-assignment2': #phần tử nào có name là atom-assignment2
        print(channel_ls[i]['id'])

C021FSDN7LJ


### Slack API: List messages trong 1 channel

In [29]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [30]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [31]:
msg_ls[21]

{'client_msg_id': 'a206160a-3b59-44d0-a800-091897d7fcc7',
 'type': 'message',
 'text': 'Chào mọi người, mình gửi assignment 1 nhaa:\n<https://www.notion.so/L-Th-o-Vy-Assignment-1-f2df526a7fb74d8a9d305061f6772cc2>\nChúc mọi người một đầu tuần vui vẻ ^^',
 'user': 'U01UMBZHU2W',
 'ts': '1619420036.000800',
 'team': 'T01B7SGGMLJ',
 'attachments': [{'service_name': "Vy's Notion on Notion",
   'title': 'Lê Thảo Vy - Assignment 1',
   'title_link': 'https://www.notion.so/L-Th-o-Vy-Assignment-1-f2df526a7fb74d8a9d305061f6772cc2',
   'text': 'Task 1: link',
   'fallback': "Vy's Notion on Notion: Lê Thảo Vy - Assignment 1",
   'from_url': 'https://www.notion.so/L-Th-o-Vy-Assignment-1-f2df526a7fb74d8a9d305061f6772cc2',
   'id': 1,
   'original_url': 'https://www.notion.so/L-Th-o-Vy-Assignment-1-f2df526a7fb74d8a9d305061f6772cc2'}],
 'blocks': [{'type': 'rich_text',
   'block_id': 'E8w3f',
   'elements': [{'type': 'rich_text_section',
     'elements': [{'type': 'text',
       'text': 'Chào mọi ngườ

In [32]:
not_learners_id = ['U01BE2PR6LU']

In [33]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 11:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 16:14:31.003600 https://github.com/danhpcv/
2021-05-10 11:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 15:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 13:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 20:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/hoaintp/
2021-05-04 00:41:18.002400 U01U6JQB695 2 ['U01UMC0N5U2', 'U01U6JQB695'] 2021-05-11 10:10:06.001300 https://github.com/Tenderriver/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [34]:
assignment_dict = {'time': [], 'user': [], 'reply_count': [], 'reply_users_count': [], 'reply_user': [], 'latest_reply': [], 'github': []}
not_learners_id = ['U01BE2PR6LU']
for i in range(40):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  assignment_dict['time'].append(ts)
  assignment_dict['user'].append(msg_ls[i]['user']) # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: assignment_dict['github'].append(github_link[0])
      if 'reply_count' in msg_ls[i].keys(): assignment_dict['reply_count'].append(msg_ls[i]['reply_count']) #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): assignment_dict['reply_users_count'].append(msg_ls[i]['reply_users_count'])
      if 'reply_users' in msg_ls[i].keys(): assignment_dict['reply_user'].append(msg_ls[i]['reply_users'])
      if 'latest_reply' in msg_ls[i].keys(): assignment_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'])))
assignment_dict

{'time': [datetime.datetime(2021, 5, 20, 23, 43, 26, 600),
  datetime.datetime(2021, 5, 20, 8, 58, 2, 300),
  datetime.datetime(2021, 5, 18, 23, 45, 9, 1300),
  datetime.datetime(2021, 5, 18, 0, 11, 11, 2600),
  datetime.datetime(2021, 5, 15, 17, 28, 4, 5600),
  datetime.datetime(2021, 5, 15, 17, 27, 40, 5500),
  datetime.datetime(2021, 5, 15, 17, 27, 20, 5400),
  datetime.datetime(2021, 5, 15, 17, 26, 41, 5300),
  datetime.datetime(2021, 5, 15, 16, 58, 42, 5000),
  datetime.datetime(2021, 5, 13, 21, 9, 48, 11000),
  datetime.datetime(2021, 5, 10, 11, 51, 46, 4700),
  datetime.datetime(2021, 5, 10, 11, 35, 39, 3800),
  datetime.datetime(2021, 5, 10, 10, 7, 35, 1200),
  datetime.datetime(2021, 5, 8, 13, 43, 25, 6900),
  datetime.datetime(2021, 5, 8, 13, 36, 43, 6000),
  datetime.datetime(2021, 5, 8, 13, 31, 59, 5200),
  datetime.datetime(2021, 5, 8, 9, 56, 13, 1700),
  datetime.datetime(2021, 5, 7, 23, 29, 48, 700),
  datetime.datetime(2021, 5, 4, 20, 55, 46, 1800),
  datetime.datetime(

In [35]:
assignment_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in assignment_dict.items() ]))
assignment_df.head()

,time,user,reply_count,reply_users_count,reply_user,latest_reply,github
0,2021-05-20 23:43:26.000600,U0226MQ73MJ,2.0,2.0,"[U01USGKQ771, U01V00KHHHP]",2021-05-15 16:14:31.003600,https://github.com/danhpcv/
1,2021-05-20 08:58:02.000300,U0226MQ73MJ,2.0,2.0,"[U01UTGS5ZNY, U01UMC08KL2]",2021-05-18 15:09:59.000800,https://github.com/auslynnguyen/
2,2021-05-18 23:45:09.001300,U0226MQ73MJ,1.0,1.0,[U01UMBX2X18],2021-05-08 13:36:43.006000,https://github.com/hoaintp/
3,2021-05-18 00:11:11.002600,U0226MQ73MJ,2.0,2.0,"[U01UMC0N5U2, U01U6JQB695]",2021-05-11 10:10:06.001300,https://github.com/Tenderriver/
4,2021-05-15 17:28:04.005600,U01BE2PR6LU,1.0,1.0,[U01UMBZHU2W],2021-05-15 00:51:26.024600,https://github.com/saturn1101/


In [36]:
import gspread
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'my_project_1.json', scope)
gc = gspread.authorize(credentials)
print('Done!')

Done!


In [38]:
# ACCESS GOOGLE SHEET
sheet_index_no = 1
spreadsheet_key = '1nNlQbhJbdfJLWCYirnHdOSB6i9KNFVPZe1qNTWkgk4o' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.add_worksheet(title= 'Assignment3', rows= "100", cols= "20") #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, assignment_df) #-> Upload assignment_df vào Sheet 2 trong Spreadsheet
print('Done!')

Done!


-------------- HẾT PHẦN OPTION 2 ---------------------